In [1]:
class GroupParams():
    pass

In [2]:
import sys
from argparse import ArgumentParser

# Create the parser
parser = ArgumentParser(description="Example Argument Parser")

# Add arguments
parser.add_argument("--no_gpu", action='store_true')
parser.add_argument("--skip_matching", action='store_true')
parser.add_argument("--source_path", "-s", required=True, type=str)
parser.add_argument("--camera", default="OPENCV", type=str)
parser.add_argument("--colmap_executable", default="", type=str)
parser.add_argument("--resize", action="store_true")
parser.add_argument("--magick_executable", default="", type=str)
parser.add_argument("--model_path", default="", type=str)

# Simulate command line input
sys.argv = ['script_name.py', '--source_path', '/path/to/source', '--camera', 'DSLR', '--resize']

# Parse arguments
args = parser.parse_args(sys.argv[1:])
# args= parser.parse_args()

# Print the type of args
# print(type(args))
# print(args)
for arg in vars(args).items():
    print(arg, type(arg))

('no_gpu', False) <class 'tuple'>
('skip_matching', False) <class 'tuple'>
('source_path', '/path/to/source') <class 'tuple'>
('camera', 'DSLR') <class 'tuple'>
('colmap_executable', '') <class 'tuple'>
('resize', True) <class 'tuple'>
('magick_executable', '') <class 'tuple'>
('model_path', '') <class 'tuple'>


In [3]:
print(vars(args).items())

dict_items([('no_gpu', False), ('skip_matching', False), ('source_path', '/path/to/source'), ('camera', 'DSLR'), ('colmap_executable', ''), ('resize', True), ('magick_executable', ''), ('model_path', '')])


In [4]:
import os
import uuid


if os.getenv('OAR_JOB_ID'):
    unique_str=os.getenv('OAR_JOB_ID')
else:
    unique_str = str(uuid.uuid4())
print(unique_str)

3b0a7715-a917-4c77-93e7-001cac67685e


# tensorboard

In [10]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [11]:
x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_model(10)
writer.flush()


# CUDA

## cuda.Event()

In [16]:
import torch

# Create CUDA events
iter_start = torch.cuda.Event(enable_timing=True)
iter_end = torch.cuda.Event(enable_timing=True)

# Example tensor operations
a = torch.randn((10_000, 10_000), device='cuda')
b = torch.randn((10_000, 10_000), device='cuda')

# Record the start event
iter_start.record()

# Perform some operations
c = torch.matmul(a, b)

# Record the end event
iter_end.record()

# Wait for the events to complete
torch.cuda.synchronize()

# Calculate the elapsed time
elapsed_time = iter_start.elapsed_time(iter_end)  # Time in milliseconds
print(f'Time elapsed for matrix multiplication: {elapsed_time} ms')


Time elapsed for matrix multiplication: 71.57247924804688 ms


# Custom tqdm progress bar

In [36]:
from tqdm import tqdm
import time, random
first_iter = 0
last_iter = 10_000
progress_bar = tqdm(range(first_iter, last_iter), desc="Training progress")

for i in range(10_000):
    time.sleep(0.0005)
    if i%10 ==0:
        progress_bar.set_postfix_str("loss"+str( random.randint(0,10)))
        progress_bar.update(10)
progress_bar.close()

Training progress: 100%|██████████| 10000/10000 [00:08<00:00, 1245.61it/s, loss5]


# Namedtuple

In [46]:
from collections import namedtuple

Point = namedtuple('Point2D', ['x', 'y'])
p = Point(x=10, y=20)
print(type(p).__name__)
print(p.x, p.y) 
print(p[0], p[1]) 
print(p[0]==p.x, p[1]==p.y)  


Point2D
10 20
10 20
True True


In [55]:
elems= "2 PINHOLE 3072 2304 2560.56 2560.56 1536 1152".split()
print(elems)
import numpy as np
for i in elems:
    print(i, type(i))   
params = np.array(tuple(map(float, elems[4:])))
print(params, type(params))
print(list(map(float, elems[4:])))

['2', 'PINHOLE', '3072', '2304', '2560.56', '2560.56', '1536', '1152']
2 <class 'str'>
PINHOLE <class 'str'>
3072 <class 'str'>
2304 <class 'str'>
2560.56 <class 'str'>
2560.56 <class 'str'>
1536 <class 'str'>
1152 <class 'str'>
[2560.56 2560.56 1536.   1152.  ] <class 'numpy.ndarray'>


In [70]:
import os
img_path= "/cs/student/projects4/ml/2023/asrivast/gaussian-splatting/output/gerrard-hall-smallest/images/IMG_2349.JPG"
print(os.path.basename(img_path))

IMG_2349.JPG
